In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# read in bli_wb2_set.csv, reza_wb2_set.csv, and farzin_wb2_set.csv from literature
bli = pd.read_csv('literature/bli_wb2_set.csv')
reza = pd.read_csv('literature/reza_wb2_set.csv')
farzin = pd.read_csv('literature/farzin_wb2_set.csv')

# bli and reza use 'T', farzin uses '1' for true
# replace with boolean
bli['exclude'] = bli['exclude'].replace({'T': True, 'F': False})
reza['exclude'] = reza['exclude'].replace({'T': True, 'F': False})
farzin['exclude'] = farzin['exclude'].replace({1: True, 0: False})

# rename to exclude_bli, exclude_reza, exclude_farzin
bli = bli.rename(columns={'exclude': 'exclude_bli'})
reza = reza.rename(columns={'exclude': 'exclude_reza'})
farzin = farzin.rename(columns={'exclude': 'exclude_farzin'})

# Title in farzin's includes weird symbols, drop it
farzin = farzin.drop(columns=['Title'])
reza = reza.drop(columns=['Title'])
# merge on 'Authors', 'Publication Type', 'SourceTitle', 'Publication Year' and 'DOI'
merged = bli.merge(reza, on=['Authors', 'Publication Type', 'SourceTitle', 'Publication Year', 'DOI']).merge(farzin, on=['Authors', 'Publication Type', 'SourceTitle', 'Publication Year', 'DOI'])


# print shape of each dataframe
print(bli.shape)
print(reza.shape)
print(farzin.shape)

print(merged.shape)
print(merged.columns)

(953, 15)
(953, 14)
(953, 14)
(953, 33)
Index(['exclude_bli', 'Title', 'Authors', 'Publication Type', 'SourceTitle',
       'Publication Year', 'Keywords_x', 'UT_x', 'DOI', 'ISSN_x', 'source_x',
       'Abstract_x', 'PMID_x', 'URL_x', 'Publication Type Other_x',
       'exclude_reza', 'Keywords_y', 'UT_y', 'ISSN_y', 'source_y',
       'Abstract_y', 'PMID_y', 'URL_y', 'Publication Type Other_y',
       'exclude_farzin', 'Keywords', 'UT', 'ISSN', 'source', 'Abstract',
       'PMID', 'URL', 'Publication Type Other'],
      dtype='object')


In [3]:
# drop columns that end in _y and _x
merged = merged.drop(columns=[c for c in merged.columns if c.endswith('_x')])
merged = merged.drop(columns=[c for c in merged.columns if c.endswith('_y')])
# move exclude_ columns to the end
exclude_columns = [c for c in merged.columns if c.startswith('exclude')]
merged = merged[[c for c in merged.columns if c not in exclude_columns] + exclude_columns]
# fill NaN with False for exclude columns
merged[exclude_columns] = merged[exclude_columns].fillna(False)
merged.columns

/var/folders/w9/kdq4wyw97jn4cg1bhrvrq8j80000gn/T/ipykernel_12850/3923723225.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged[exclude_columns] = merged[exclude_columns].fillna(False)


Index(['Title', 'Authors', 'Publication Type', 'SourceTitle',
       'Publication Year', 'DOI', 'Keywords', 'UT', 'ISSN', 'source',
       'Abstract', 'PMID', 'URL', 'Publication Type Other', 'exclude_bli',
       'exclude_reza', 'exclude_farzin'],
      dtype='object')

In [4]:
# how many are excluded by each source
print(f'bli: {merged[merged["exclude_bli"]].shape[0]}')
print(f'reza: {merged[merged["exclude_reza"]].shape[0]}')
print(f'farzin: {merged[merged["exclude_farzin"]].shape[0]}')

bli: 740
reza: 175
farzin: 277


In [5]:
# how many rows are excluded by all three?
# print(merged[(merged['exclude_bli'] == True) & (merged['exclude_reza'] == True) & (merged['exclude_farzin'] == True)].shape)

# create new column 'by_all' that is True if all exclude columns are True
merged['by_all'] = merged[exclude_columns].all(axis=1)
# rows excluded by both bli and reza
merged['by_bli_reza'] = (merged['exclude_bli'] == True) & (merged['exclude_reza'] == True)
# rows excluded by both bli and farzin
merged['by_bli_farzin'] = (merged['exclude_bli'] == True) & (merged['exclude_farzin'] == True)
# rows excluded by both reza and farzin
merged['by_reza_farzin'] = (merged['exclude_reza'] == True) & (merged['exclude_farzin'] == True)

# print counts
print(merged['by_all'].value_counts())
# drop rows excluded by all three
merged = merged[merged['by_all'] == False]
# drop columns
merged = merged.drop(columns=['by_all'])

print(f'Size: {merged.shape}, By bli and reza: {merged["by_bli_reza"].value_counts()}')
merged = merged[merged['by_bli_reza'] == False]
merged = merged.drop(columns=['by_bli_reza'])

print(f'Size: {merged.shape}, By bli and farzin: {merged["by_bli_farzin"].value_counts()}')
merged = merged[merged['by_bli_farzin'] == False]
merged = merged.drop(columns=['by_bli_farzin'])

print(f'Size: {merged.shape}, By reza and farzin: {merged["by_reza_farzin"].value_counts()}')
merged = merged[merged['by_reza_farzin'] == False]
merged = merged.drop(columns=['by_reza_farzin'])

print(merged.shape)

by_all
False    866
True      87
Name: count, dtype: int64
Size: (866, 20), By bli and reza: by_bli_reza
False    791
True      75
Name: count, dtype: int64
Size: (791, 19), By bli and farzin: by_bli_farzin
False    623
True     168
Name: count, dtype: int64
Size: (623, 18), By reza and farzin: by_reza_farzin
False    619
True       4
Name: count, dtype: int64
(619, 17)


In [6]:
# how many are excluded by each source
print(f'bli: {merged[merged["exclude_bli"]].shape[0]}')
print(f'reza: {merged[merged["exclude_reza"]].shape[0]}')
print(f'farzin: {merged[merged["exclude_farzin"]].shape[0]}')

bli: 410
reza: 9
farzin: 18


In [7]:
# to csv
merged.to_csv('literature/merged1.csv', index=False)